In [ ]:
import numpy as np
import pandas as pd
import yaml
import json
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import model_from_json

from tqdm import tqdm
import os
import sys
import warnings
from dotenv import load_dotenv

warnings.filterwarnings("ignore")
load_dotenv()
REPO_PATH = os.getenv("REPO_PATH")
sys.path.insert(0, rf'{REPO_PATH}src_HF')

from utils.forecast_utils import RNNGenerator, ForecastModel

### Load data

In [ ]:
# Specify data
gen = RNNGenerator(future='CLc1', topic='CRU')

# load yaml file
with open('variable_config.yaml') as file:
    config = yaml.load(file, Loader=yaml.FullLoader)

# Define feature list and target
feature_columns = list()
feature_columns += config['BASE']
feature_columns += config['SENTIMENT']
feature_columns += config['TEMPORAL']
feature_columns += config['LAGS']

target_column = 'TARGET'

# Define window size
window_size = 30


### Load models

In [ ]:
MODEL_NAMES = [
    'CLc1_CRU_GRU_2024.05.15_12.46',
    'CLc1_CRU_LSTM_2024.05.15_13.11',
    'CLc1_CRU_BiLSTM_2024.05.15_13.24',
    'CLc1_CRU_BiGRU_2024.05.15_13.27'
]

model_params = {
    'units_first_layer': 64,
    'units_second_layer': 64,
    'dropout_rate_first': 0.1,
    'dropout_rate_second': 0.1,
    'l2_strength': 1e-05,
    'learning_rate': 0.01,
    'batch_size': 32,
    'noise_std': 0.01
}

data_params = {
    'feature_columns': feature_columns,
    'target_column': target_column,
    'window_size': window_size,
    'test_size': 0.2,
    'val_size': 0.2
}

model_dict = dict()
for model_name in tqdm(MODEL_NAMES):
    model_dict[model_name] = ForecastModel(model_name, model_params, data_params)



In [ ]:
for model in model_dict.values():
    model.describe()
    print()

In [ ]:
# Evaluate the model
view = 700

fig, ax = plt.subplots(figsize=(10, 5), dpi=200)

for i, model in enumerate(model_dict.values()):
    if i == 0:
        actual = model.test_targets[-view:]
        ax.plot(actual, label='Actual ' + model.model_name.split('_')[0], color='gray', lw=0.7)

    ax.plot(model.test_predictions[-view:], label=' '.join(model.model_name.split('_')[1:3]), lw=0.7)
    
ax.set_title('Model Fit vs Actual')
ax.set_xlabel('Samples')
ax.set_ylabel('Price')
ax.legend(frameon=False)
ax.grid(alpha=0.3)